In [3]:
from pathlib import Path
import xml.etree.ElementTree as ET
import json
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import os, shutil
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF

#from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data.dataset import Dataset
from torchvision import models, transforms
from collections import defaultdict
from random import shuffle
from PIL import Image
from tqdm import tqdm_notebook
from shapely.geometry import Polygon


import functools

In [4]:
def getDictFromXml(filePath):
    tree = ET.parse(filePath)
    root = tree.getroot()
    
    parking = root.attrib
    parking['spaces'] = []
    for space in root.findall('space'):
        sp = {}
        sp['id'] = space.attrib['id']
        
        sp['occupied'] = space.attrib.get('occupied', 0)
        sp['rotatedRect'] = {}

        for point in space.find('rotatedRect'):
            sp['rotatedRect'][point.tag] = point.attrib

        sp['contour'] = []

        for point in space.find('contour'):
            sp['contour'].append(point.attrib)
            
        parking['spaces'].append(sp)
    
    return parking
    

In [5]:
def show_img(image, figsize=None, ax=None):
    if not ax: 
        fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(np.transpose(image, (1, 2, 0)), zorder=1)
    ax.grid()
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    return ax

def to_corner(cx, cy, w, h, a, up, right):
    theta = a.double() * np.pi / 180 
    # cx, cy - center of square coordinates
    # x, y - coordinates of a corner point of the square
    # theta is the angle of rotation
    if up:
        y = cy + (h / 2)
    else:
        y = cy - (h / 2)

    if right:
        x = cx + (w / 2)
    else:
        x = cx - (w / 2)

    # translate point to origin
    temp_x = x - cx
    temp_y = y - cy

    # now apply rotation
    rotated_x = temp_x * np.cos(theta) - temp_y * np.sin(theta)
    rotated_y = temp_x * np.sin(theta) + temp_y * np.cos(theta)

    # translate back
    x_prime = rotated_x + cx
    y_prime = rotated_y + cy
    
    return x_prime, y_prime

def create_polygon(cx, cy, w, h, a):
    return Polygon([to_corner(cx, cy, w, h, a, up, right) for up, right in 
                    [(True, False), (True, True), (False, True), (False, False)]])

def create_polygon_list(rotated_bboxes):
    return [create_polygon(box[0], box[1], box[3], box[2], box[4]) for box in rotated_bboxes]

def draw_poly(ax, poly, c='black', linewidth=2):
    x,y = poly.exterior.xy
    ax.plot(x, y, color=c, alpha=0.7, linewidth=linewidth, solid_capstyle='round', zorder=2)

def show_truth(image, rotated_bboxes, labels, ax, linewidth=1):
    polygons = create_polygon_list(rotated_bboxes)
    labels = [int(num) for num in labels.tolist()]
    show_img(image, ax=ax)
    
    legend_elements = [Patch(edgecolor='g', label='Occupied', fill=False), 
                       Patch(edgecolor='r', label='Empty', fill=False)]
    ax.legend(handles=legend_elements, loc='upper right')
    
    color_map = {0: 'r', 1: 'g'}

    for i, poly in enumerate(polygons):
        draw_poly(ax, poly, c=color_map[labels[i]], linewidth=linewidth)

In [6]:
parking_lots = ["PUCPR"]
# "PUCPR": (192, 492, 353, 1170)
# assuming all pictures have same size crop
parking_crop = {"PUCPR": [(353, 954, 192, 492)]}


for bdpk in parking_lots:
    pic_paths = Path().glob(f"PKLot/AugPKLot/{bdpk}/*/*/*.jpg")
    pic_paths = list(pic_paths)
    print('# of images: ', len(pic_paths))
    
    datalist = {'image_path': [], 'occupied': [], 'rotated_bbox': []}
    
    for picp in tqdm_notebook(pic_paths):
        
        try:
            xmlp = picp.with_suffix('.xml')
            spaces = getDictFromXml(xmlp)['spaces']
        except:
            print('File not found!')
            continue
    
        rotated_bbox = []
        occupied = []
        for sp in spaces:
            rbox = (int(sp['rotatedRect']['center']['x']),
                    int(sp['rotatedRect']['center']['y']),
                    int(sp['rotatedRect']['size']['h']),
                    int(sp['rotatedRect']['size']['w']),
                    int(sp['rotatedRect']['angle']['d']))
            rotated_bbox.append(rbox)
            occupied.append(int(sp['occupied']))
    
    

        pic = plt.imread(picp)
        mypic = Image.open(picp)
        
        
        pklot_crop = parking_crop[bdpk]
        num_crops = len(pklot_crop)
        size_x = pklot_crop[0][1] - pklot_crop[0][0]
        size_y = pklot_crop[0][3] - pklot_crop[0][2]

        cropped_pics = np.zeros((size_x, size_y, 3), dtype=np.int)

        for i, (x1, x2, y1, y2) in enumerate(pklot_crop):
            if pic.shape[0] == 300:
                pass
            else:
                cropped_pics = pic[y1:y2, x1:x2, :]
                plt.imsave(picp, cropped_pics)

            crop_spaces = []
            crop_occp = []
            for i, (cx,cy,h,w,a) in enumerate(rotated_bbox):
                if x1 < cx and cx < x2 and y1 < cy and cy < y2:
                    crop_occp.append(occupied[i])
                    crop_spaces.append([cx - x1,cy - y1,h,w,a])
        
#         ax = show_img(TF.to_tensor(mypic))
#         for poly in create_polygon_list(torch.Tensor(crop_spaces)):
#             draw_poly(ax, poly)
        
#         plt.show()
        
        occupied = ' '.join(map(lambda x: str(x), crop_occp))
        rotated_bbox = ' '.join(map(lambda x: ' '.join(map(lambda y: str(y), x)), crop_spaces))
        
        
        datalist['image_path'].append(picp)
        datalist['occupied'].append(occupied)
        datalist['rotated_bbox'].append(rotated_bbox)
        
        
    
    
    
    data = pd.DataFrame({'image_path': datalist['image_path'], 
                         'occupied': datalist['occupied'],
                         'rotated_bbox': datalist['rotated_bbox']
                        })


# of images:  4474


File not found!



In [7]:
data.head()

,image_path,occupied,rotated_bbox
0,PKLot/AugPKLot/PUCPR/Rainy/2012-11-10/2012-11-...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,13 16 32 52 -77 45 15 36 54 -79 77 18 31 50 -7...
1,PKLot/AugPKLot/PUCPR/Rainy/2012-11-10/2012-11-...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,13 16 32 52 -77 45 15 36 54 -79 77 18 31 50 -7...
2,PKLot/AugPKLot/PUCPR/Rainy/2012-11-10/2012-11-...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,13 16 32 52 -77 45 15 36 54 -79 77 18 31 50 -7...
3,PKLot/AugPKLot/PUCPR/Rainy/2012-11-10/2012-11-...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,13 16 32 52 -77 45 15 36 54 -79 77 18 31 50 -7...
4,PKLot/AugPKLot/PUCPR/Rainy/2012-11-10/2012-11-...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,13 16 32 52 -77 45 15 36 54 -79 77 18 31 50 -7...


In [8]:
print(len(datalist['image_path']))

4473


In [9]:
data.shape

(4473, 3)

In [10]:
data.to_csv('PUCPR_cropped.csv', index=False)